#**Weather Python Project**

In [5]:
# Weather Program with 5-Day Forecast (Fixed Version)
import requests
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime

class WeatherApp:
    def __init__(self):
        # API configuration
        self.API_KEY = "cf7485a23cb44d25923175752253003"  # Free tier key (replace with your own)
        self.BASE_URL = "https://www.weatherapi.com/my/"
        self.FORECAST_URL = "https://www.weatherapi.com/my/"

        # Create widgets
        self.city_input = widgets.Text(
            value="New York",
            placeholder="Enter city name",
            description="City:",
            style={'description_width': 'initial'}
        )
        self.country_input = widgets.Text(
            value="US",
            placeholder="Country code (e.g. US)",
            description="Country:",
            style={'description_width': 'initial'}
        )
        self.unit_selector = widgets.Dropdown(
            options=[('Celsius', 'metric'), ('Fahrenheit', 'imperial'), ('Kelvin', 'standard')],
            value='metric',
            description='Units:'
        )
        self.search_btn = widgets.Button(description="Get Weather", button_style='success')
        self.output_area = widgets.Output()

        # Set up event handlers
        self.search_btn.on_click(self.get_weather)

        # Display initial UI
        self.display_ui()

    def display_ui(self):
        """Display the main user interface"""
        clear_output()
        display(widgets.VBox([
            widgets.HTML("<h2>⛅ Weather Dashboard</h2>"),
            widgets.HTML("Enter location and units:"),
            widgets.HBox([self.city_input, self.country_input]),
            self.unit_selector,
            self.search_btn,
            self.output_area
        ]))

    def get_weather(self, button):
        """Fetch weather data from API"""
        city = self.city_input.value
        country = self.country_input.value
        units = self.unit_selector.value

        with self.output_area:
            clear_output()

            try:
                # Current weather
                current_params = {
                    'q': f"{city},{country}",
                    'appid': self.API_KEY,
                    'units': units
                }
                current_response = requests.get(self.BASE_URL, params=current_params).json()

                # 5-day forecast
                forecast_params = {
                    'q': f"{city},{country}",
                    'appid': self.API_KEY,
                    'units': units
                }
                forecast_response = requests.get(self.FORECAST_URL, params=forecast_params).json()

                # Display current weather
                self.display_current_weather(current_response, units)

                # Display forecast
                self.display_forecast(forecast_response, units)

            except Exception as e:
                print(f"❌ Error fetching weather data: {str(e)}")
                print("Please check your city/country and try again")

    def display_current_weather(self, data, units):
        """Display current weather conditions"""
        if data.get('cod') != 200:
            print(f"Error: {data.get('message', 'Unknown error')}")
            return

        temp_unit = '°C' if units == 'metric' else '°F' if units == 'imperial' else 'K'
        wind_unit = 'm/s' if units == 'metric' else 'mph' if units == 'imperial' else 'm/s'

        city = data['name']
        country = data['sys']['country']
        temp = data['main']['temp']
        feels_like = data['main']['feels_like']
        humidity = data['main']['humidity']
        wind_speed = data['wind']['speed']
        description = data['weather'][0]['description'].title()
        icon = data['weather'][0]['icon']
        sunrise = datetime.fromtimestamp(data['sys']['sunrise']).strftime('%H:%M')
        sunset = datetime.fromtimestamp(data['sys']['sunset']).strftime('%H:%M')

        # Display current weather
        print(f"📍 {city}, {country}")
        print(f"🌡 Current: {temp}{temp_unit} (Feels like {feels_like}{temp_unit})")
        print(f"🌤 {description}")
        print(f"💧 Humidity: {humidity}%")
        print(f"🌬 Wind: {wind_speed} {wind_unit}")
        print(f"🌅 Sunrise: {sunrise}  🌇 Sunset: {sunset}")

        # Display weather icon
        icon_url = f"http://openweathermap.org/img/wn/{icon}@2x.png"
        display(widgets.Image(value=requests.get(icon_url).content))

    def display_forecast(self, data, units):
        """Display 5-day forecast with matplotlib"""
        if data.get('cod') != '200':
            return

        temp_unit = '°C' if units == 'metric' else '°F' if units == 'imperial' else 'K'

        # Process forecast data
        dates = []
        temps = []
        descriptions = []

        for forecast in data['list'][:8]:  # Next 24 hours (3-hour intervals)
            timestamp = datetime.fromtimestamp(forecast['dt'])
            dates.append(timestamp.strftime('%a %H:%M'))
            temps.append(forecast['main']['temp'])
            descriptions.append(forecast['weather'][0]['description'].title())

        # Create forecast plot
        plt.figure(figsize=(10, 5))
        plt.plot(dates, temps, marker='o', linestyle='-', color='red')
        plt.title(f"24-Hour Forecast ({temp_unit})")
        plt.xlabel("Time")
        plt.ylabel(f"Temperature ({temp_unit})")
        plt.xticks(rotation=45)
        plt.grid(True)

        # Add weather descriptions
        for i, desc in enumerate(descriptions):
            plt.annotate(desc, (dates[i], temps[i]), textcoords="offset points", xytext=(0,10), ha='center')

        plt.tight_layout()
        plt.show()

# Start the weather app
weather_app = WeatherApp()